# Tarea 1 - Pregunta 1

## Estrategia
Se quiere obtener la llave con la que se encriptó un mensaje con OTP. La estrategia para lograrlo consiste en utilizar la estrategia vista en clases para un largo específico encontrar la mejor llave posible, la cual se basa en calcular la distancia en base a la probabilidad de aparición de los caracteres en el idioma inglés. Entonces, utilizando esa estrategia para cada largo dentro del rango esperado que es (1, len(ciphertext) // 50), se obtiene una lista con las mejores llaves para su respectivo largo. Con esto, se deja la llave que logra decriptar el texto tal que tenga la menor distancia respecto al diccionario de frecuencias, por lo que se parece más al idioma esperado.

In [1]:
from typing import Callable
import time
"""
Diccionario de frecuencias, que también define el alfabeto sobre el cual se trabaja
"""
FRECUENCIES = {
    'A': 0.082, 'B': 0.015, 'C': 0.027, 'D': 0.047, 'E': 0.13, 'F': 0.022, 'G': 0.02,
    'H': 0.062, 'I': 0.069, 'J': 0.0016, 'K': 0.0081, 'L': 0.04, 'M': 0.027, 'N': 0.067,
    'O': 0.078, 'P': 0.019, 'Q': 0.0011, 'R': 0.059, 'S': 0.062, 'T': 0.096,
    'U': 0.027, 'V': 0.0097, 'W': 0.024, 'X': 0.0015, 'Y': 0.02, 'Z': 0.00078
}

def encrypt(text, key):
    """
    Función de encriptación OTP para test de la solución
    """
    key_len = len(key)
    alphabet = list(FRECUENCIES.keys())
    encrypted = ""
    key_index = 0
    for char in text:
        encrypted += alphabet[(alphabet.index(char) + alphabet.index(key[key_index])) % len(alphabet)]
        key_index += 1
        if key_index > key_len - 1: key_index = 0

    return encrypted

def decrypt(text, key):
    """
    Función de decriptación OTP para test de solución y para calcular la distancia de un string sobre
    las frecuencias del lenguaje dado
    """
    key_len = len(key)
    alphabet = list(FRECUENCIES.keys())
    decrypted = ""
    key_index = 0
    for char in text:
        decrypted += alphabet[(alphabet.index(char) - alphabet.index(key[key_index])) % len(alphabet)]
        key_index += 1
        if key_index > key_len - 1: key_index = 0

    return decrypted

def abs_distance(string: str, frequencies: {str: float}) -> float:
    """
    Función de distancia dada en el enunciado para probar la solución
    """
    return sum([
        abs(frequencies[c] - string.count(c) / len(string))
        for c in frequencies
    ])

def most_likely_char(pos: int, cipher: str, frequencies: dict, key_length: int, distance: Callable[[str, dict], float]) -> (str, float):
    """
    Obtiene el mejor caracter de la llave en base a una función de distancia "distance" y el diccionario de frecuencias.
    """
    to_decrypt = [
        cipher[i * key_length + pos]
        for i in range(len(cipher) // key_length)
    ]
    
    alphabet = list(frequencies.keys())
    best_char = 'A'
    best_distance = len(alphabet)
    
    for candidate in alphabet:
        decrypted = ""
        n_candidate = alphabet.index(candidate)
        for c in to_decrypt:
            n_c = alphabet.index(c)
            n = (n_c - n_candidate) % len(alphabet)
            decrypted += alphabet[n]

        diff = distance(decrypted, frequencies)

        if diff < best_distance:
            best_char = candidate
            best_distance = diff

    return (best_char, best_distance)

def break_rp(ciphertext: str, frequencies: {str: float}, distance: Callable[[str, dict], float]) -> str:
    """
    Para cada posible largo de la llave, calcula la mejor llave en base a la función de distancia "distance"
    Retorna la llave que tenía la mejor distancia obtenida
    """
    # Se almacena la mejor llave, es decir, la que tiene la menor distancia respecto al alfabeto
    best_key = ""
    best_distance = 100

    # Para cada largo de llave entre 1 y ciphertext / 50
    for k in range(1, len(ciphertext)//50 + 1):
        probable_key = ""
        probable_key_distance = 0
        for i in range(k):
            key_char, key_distance = most_likely_char(i, ciphertext, frequencies, k, distance) # Obtiene la mejor llave con su distancia para el largo de llave k
            probable_key += key_char
            
        probable_key_distance = distance(decrypt(ciphertext, probable_key), frequencies)

        if probable_key_distance < best_distance:
            best_key = probable_key
            best_distance = probable_key_distance
        
    return best_key

def upper_clean_input(text):
    """
    Función que limpia el texto y lo deja en el formato requerido para utilizar break_otp
    Recibe un texto y lo retorna en uppercase sin símbolos externos
    """
    for i in range(10):
        text = text.replace(str(i), "")
        
    symbols = [" ", ",", ".", ":", ";", "-", "_", "%", "#", "$", "&", "/", "(", ")", "'\'", "?", "!", "?", "¿", "'"]
    
    for symbol in symbols:
        text = text.replace(symbol, "")
    return text.upper()


In [5]:
a = "A talismanic shirt is an item of clothing worn as a talisman. Talismanic shirts are found throughout the Islamic world, and can be grouped into four general types that differ in style and the symbols used: Ottoman, Safavid, Mughal, and West African. Such shirts were believed to be capable of offering protection to the wearer, especially in battle. This 17th-century Turkish talismanic shirt is made of cotton and inscribed with Quranic verses, the names of Allah, Islamic prayers, and views of Mecca and Medina in ink and gold. The shirt forms part of the Khalili Collection of Hajj and the Arts of Pilgrimage."
text = upper_clean_input(a*10)[:2500]
#key = upper_clean_input("criptografia y seguridad computacional")
key = upper_clean_input("zjhsakqqdwdqajlxcnkas")
ciphertext = encrypt(text, key)

In [6]:
i = time.time()
probable_key = break_rp(ciphertext, FRECUENCIES, abs_distance)
f = time.time()
print(probable_key)
if probable_key == key.upper():
    print("Success")
else:
    print("Failed:", "Expected:", key, "/", "Recieved:", probable_key)
print(f"Time from start to end: {f - i} seconds")

ZJHSAKQQDWDQAJLXCNKAS
Success
Time from start to end: 2.1825003623962402 seconds
